# Camada Gold


In [0]:
# Importação de bibliotecas
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import timedelta
from pyspark.sql.types import DecimalType


In [0]:
# Definição do caminho da pasta Gold
catalogo = "medalhao"
gold_db_name = "gold"

## 1º Projeto — Área de Logística (Vendas por Localidade)

###1.1 Criação da tabela gold.ft_vendas_consumidor_local

In [0]:
# Leitura das tabelas da silver
ft_pedido_total_silver_df = spark.table("medalhao.silver.ft_pedido_total")
ft_consumidores_silver_df = spark.table("medalhao.silver.ft_consumidores")

In [0]:
# Checando se cada consumidor possui somente um registro de pedido
display(ft_pedido_total_silver_df.groupBy("id_consumidor").count().orderBy(F.desc("count")).limit(5))

id_consumidor,count
d3e3cb2b2922f53b23b090d9668a1b13,1
3f0bd16414d5fee5cb1d19769c3c825e,1
167b9485947ed0a354a3f8dad04eb199,1
4d1e62429d88723ea3c9a72fb4c4ffe3,1
96d3328d8b8625bc4d656279e1515bee,1


In [0]:
# Join da tabela ft_pedido_total_silver_df com ft_consumidores_silver_df
ft_vendas_consumidor_local_df = (
    ft_pedido_total_silver_df
    .join(
    ft_consumidores_silver_df,
    on = "id_consumidor",
    how = "inner"
    )
    .select(
    F.col("id_pedido"),
    F.col("id_consumidor"),
    F.col("valor_total_pago_brl").alias("valor_total_pedido_brl").cast(DecimalType(12,2)),
    F.col("cidade"),
    F.col("estado"),
    F.col("data_pedido").cast("date")
    )
)
    
display(ft_vendas_consumidor_local_df.limit(5))

# Salvando a tabela na gold
ft_vendas_consumidor_local_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{gold_db_name}.ft_vendas_consumidor_local")

id_pedido,id_consumidor,valor_total_pedido_brl,cidade,estado,data_pedido
f4471dae8c482f51aa1826cd9f5d4433,167b9485947ed0a354a3f8dad04eb199,232.14,BARUERI,SP,2018-07-05
377627ea55b004e8d55a2af17ba3c0e1,3f0bd16414d5fee5cb1d19769c3c825e,207.26,SAO PAULO,SP,2018-05-20
36da3594f270f9d28bae07df5974d240,4d1e62429d88723ea3c9a72fb4c4ffe3,164.54,QUATIS,RJ,2017-10-21
67009d1f213f762960c97b4f8629159c,d3e3cb2b2922f53b23b090d9668a1b13,216.83,CANARANA,MT,2018-01-20
0ac372f92fbec55ca27ef4fa99a6a13a,96d3328d8b8625bc4d656279e1515bee,23.10,RIO DE JANEIRO,RJ,2018-01-11


###1.2 Criação da view gold.view_total_compras_por_consumidor


In [0]:
%sql
--Agrupando por localidade : estado e cidade
CREATE OR REPLACE VIEW medalhao.gold.view_total_compras_por_consumidor AS
SELECT 
    estado,
    cidade,
    COUNT(*) AS quantidade_vendas,
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM medalhao.gold.ft_vendas_consumidor_local
GROUP BY estado, cidade
ORDER BY estado;

SELECT *
FROM medalhao.gold.view_total_compras_por_consumidor
LIMIT 5;



estado,cidade,quantidade_vendas,valor_total_localidade
AC,MANOEL URBANO,1,248.71
AC,BRASILEIA,1,139.53
AC,RIO BRANCO,70,16928.47
AC,XAPURI,2,445.89
AC,SENADOR GUIOMARD,2,547.09


**Consulta SQL:** Retorna o total de vendas por estado

In [0]:
%sql
-- A consulta foi ordenada por total_vendas_estado para visualizar os estados com mais vendas
SELECT  estado, 
        SUM(quantidade_vendas) as total_vendas_estado
FROM medalhao.gold.view_total_compras_por_consumidor
GROUP BY estado
-- Ordenando por total_vendas_estado para visualizar os estados com mais vendas primeiro
ORDER BY total_vendas_estado DESC;


estado,total_vendas_estado
SP,41745
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033
GO,2020


> ##2º Projeto — Área de Logística (Análise de Atrasos de Entregas)

### 2.1 Criação da tabela gold.ft_atrasos_pedidos_local_vendedor


In [0]:
# Leitura das tabelas da silver
ft_pedidos_silver_df = spark.table("medalhao.silver.ft_pedidos")
ft_itens_pedidos_silver_df = spark.table("medalhao.silver.ft_itens_pedidos")


In [0]:
# Cada pedido está associado a um consumidor, mas um pedido possui vários itens
ft_itens_pedidos_silver_df.groupBy("id_pedido").count().orderBy("count", ascending=False).limit(5).display()

id_pedido,count
8272b63d03f5f79c56e9e4120aec44ef,21
1b15974a0141d54e36626dca3fdc731a,20
ab14fdcfbe524636d65ee38360e22ce8,20
428a2f660dc84138d969ccd69a0ab6d5,15
9ef13efd6949e4573a18964dd1bbe7f5,15


In [0]:
# Join da tabela ft_pedidos_silver_df com ft_consumidores_silver_df e ft_itens_pedidos_silver_df
ft_atrasos_pedidos_local_vendedor_df = (
    ft_pedidos_silver_df
    .join(
    ft_consumidores_silver_df,
    on = "id_consumidor",
    how = "inner"
    )
    .join(
    ft_itens_pedidos_silver_df,
    on = "id_pedido",
    how = "inner"
    )
    .select(
    F.col("id_pedido"),
    F.col("id_vendedor"),
    F.col("id_consumidor"),
    F.col("entrega_no_prazo"),
    F.col("tempo_entrega_dias"),
    F.col("tempo_entrega_estimado_dias"),
    F.col("cidade"),
    F.col("estado")
    )
    
)
display(ft_atrasos_pedidos_local_vendedor_df.limit(5))

# Salva a tabela na gold
ft_atrasos_pedidos_local_vendedor_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor")

id_pedido,id_vendedor,id_consumidor,entrega_no_prazo,tempo_entrega_dias,tempo_entrega_estimado_dias,cidade,estado
e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,9ef432eb6251297304e76186b10a928d,Sim,8,16,SAO PAULO,SP
53cdb2fc8bc7dce0b6741e2150273451,289cdb325fb7e7f891c38608bf9e0962,b0830fb4747a6c6d20dea0b8c802d7ef,Sim,14,20,BARREIRAS,BA
47770eb9100c2d0c44946d9cf07ec65d,4869f7a5dfa277a7dca6462dcf3b52b2,41ce2a54c0b03bf3443c3d931a367089,Sim,9,27,VIANOPOLIS,GO
949d5b44dbf5de918fe9c16f97b45f8a,66922902710d126a0e7d26b0e3805106,f88197465ea7920adcdbec7375364d82,Sim,14,27,SAO GONCALO DO AMARANTE,RN
ad21c59c0840e6cb83a9ceb5573f8159,2c9e548be18521d1c43cde1c582c6de8,8ab97904e6daea8866dbdbc4fb7aad2c,Sim,3,13,SANTO ANDRE,SP


###2.2 Criação das Views Analíticas

####2.2.1 gold.view_tempo_medio_entrega_localidade

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_tempo_medio_entrega_localidade AS
SELECT
    estado,
    cidade,

    -- Como o tempo de entrega é em dias, a média deve ser um inteiro
    ROUND(AVG(tempo_entrega_dias), 0) AS tempo_medio_entrega,
    ROUND(AVG(tempo_entrega_estimado_dias), 0) AS tempo_medio_estimado,

    -- Se a maioria dos pedidos não atrasa, então NÃO, caso contrário SIM
    CASE 
        WHEN SUM(CASE WHEN entrega_no_prazo = 'Sim' THEN 1 ELSE 0 END) >
             SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END)
            THEN 'NÃO'
        ELSE 'SIM'
    END AS entrega_maior_que_estimado

FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY estado, cidade
ORDER BY estado, cidade;

SELECT *
FROM medalhao.gold.view_tempo_medio_entrega_localidade
LIMIT 5;


    

estado,cidade,tempo_medio_entrega,tempo_medio_estimado,entrega_maior_que_estimado
AC,BRASILEIA,30.0,41.0,NÃO
AC,CRUZEIRO DO SUL,24.0,57.0,NÃO
AC,EPITACIOLANDIA,13.0,44.0,NÃO
AC,MANOEL URBANO,12.0,46.0,NÃO
AC,PORTO ACRE,29.0,43.0,NÃO


#### 2.2.2 gold.view_vendedor_pontualidade


In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendedor_pontualidade AS
SELECT
    id_vendedor,
    -- total_pedidos_entregues (sem incluir "Não Entregue")
    COUNT(DISTINCT CASE 
        WHEN entrega_no_prazo != 'Não Entregue' THEN id_pedido 
        END
    ) AS total_pedidos_entregues,

    -- total_pedidos
    COUNT(DISTINCT id_pedido) AS total_pedidos,

    -- total_atrasados (entrega_no_prazo = 'Não')
    SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados,

    -- percentual_atraso = (total_atrasados / total_pedidos_entregues) * 100
    ROUND(
        -- Necessário usar TRY_DIVIDE para evitar divisão por zero
        TRY_DIVIDE(
            SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END),
            COUNT(DISTINCT CASE 
                WHEN entrega_no_prazo != 'Não Entregue' THEN id_pedido 
                END
            )
        ) * 100,
    2) AS percentual_atraso

FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor;

SELECT *
FROM medalhao.gold.view_vendedor_pontualidade
LIMIT 5;


id_vendedor,total_pedidos_entregues,total_pedidos,total_atrasados,percentual_atraso
16090f2ca825584b5a147ab24aa30c86,398,402,32,8.04
36a968b544695394e4e9d7572688598f,148,154,5,3.38
b410bdd36d5db7a65dcd42b7ead933b8,42,44,0,0.0
7e3f87d16fb353f408d467e74fbd8014,91,92,6,6.59
74c7dec0a384d8a05950e629bd23bde9,4,5,0,0.0


## 3º Projeto — Área Comercial (Análises de Vendas por Período)


### 3.1 Criação da Dimensão de Tempo — gold.dm_tempo


In [0]:
# mesma data definida para cotação
data_inicio = "2016-09-03"
data_fim = "2018-10-16"

In [0]:
dicionario_mes = {
    "January": "Janeiro",
    "February": "Fevereiro",
    "March": "Março",
    "April": "Abril",
    "May": "Maio",
    "June": "Junho",
    "July": "Julho",
    "August": "Agosto",
    "September": "Setembro",
    "October": "Outubro",
    "November": "Novembro",
    "December": "Dezembro"
}

dicionario_semana = {
    "Monday": "Segunda",
    "Tuesday": "Terça",
    "Wednesday": "Quarta",
    "Thursday": "Quinta",
    "Friday": "Sexta",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

# Cria o mapeamento para português
mapa_meses = F.create_map([F.lit(x) for x in sum(dicionario_mes.items(), ())])
mapa_semana = F.create_map([F.lit(x) for x in sum(dicionario_semana.items(), ())])

In [0]:
dm_tempo_df = (
    spark.sql("SELECT sequence(to_date('{0}'), to_date('{1}'), interval 1 day) AS datas"
              .format(data_inicio, data_fim))
    .withColumn("sk_tempo", F.explode("datas"))
    .drop("datas")
    .withColumn("ano", F.year("sk_tempo"))
    .withColumn("trimestre", F.quarter("sk_tempo"))
    .withColumn("mes", F.month("sk_tempo"))
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn("dia", F.dayofmonth("sk_tempo"))
    .withColumn("mes_nome_en", F.date_format("sk_tempo", "MMMM"))  
    .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))  # 1-> Domingo
    .withColumn("dia_da_semana_nome_en", F.date_format("sk_tempo", "EEEE"))  
    # Para manter a consistência com os dados em português
    .withColumn("mes_nome", mapa_meses[F.col("mes_nome_en")])
    .withColumn("dia_da_semana_nome", mapa_semana[F.col("dia_da_semana_nome_en")])
    .withColumn(
        "eh_fim_de_semana",
        F.when(F.col("dia_da_semana_num").isin(1, 7), "Sim").otherwise("Não")
    )
    .drop("dia_da_semana_nome_en", "mes_nome_en")
    
)

display(dm_tempo_df.limit(10))
dm_tempo_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{gold_db_name}.dm_tempo")


sk_tempo,ano,trimestre,mes,semana_do_ano,dia,dia_da_semana_num,mes_nome,dia_da_semana_nome,eh_fim_de_semana
2016-09-03,2016,3,9,35,3,7,Setembro,Sábado,Sim
2016-09-04,2016,3,9,35,4,1,Setembro,Domingo,Sim
2016-09-05,2016,3,9,36,5,2,Setembro,Segunda,Não
2016-09-06,2016,3,9,36,6,3,Setembro,Terça,Não
2016-09-07,2016,3,9,36,7,4,Setembro,Quarta,Não
2016-09-08,2016,3,9,36,8,5,Setembro,Quinta,Não
2016-09-09,2016,3,9,36,9,6,Setembro,Sexta,Não
2016-09-10,2016,3,9,36,10,7,Setembro,Sábado,Sim
2016-09-11,2016,3,9,36,11,1,Setembro,Domingo,Sim
2016-09-12,2016,3,9,37,12,2,Setembro,Segunda,Não


### 3.2 Criação da Fato gold.ft_vendas_geral


In [0]:
ft_produtos_silver_df = spark.table("medalhao.silver.ft_produtos")
ft_avaliacoes_pedidos_silver_df = spark.table("medalhao.silver.ft_avaliacoes_pedidos")
dm_cotacao_dolar_silver_df = spark.table("medalhao.silver.dm_cotacao_dolar")

In [0]:
# Transformação dos tipos
dm_cotacao_dolar_silver_df = (
    dm_cotacao_dolar_silver_df
        .withColumn("data", F.to_date("data"))
        .withColumn("cotacao_dolar", F.col("cotacao_dolar").cast(DecimalType(8,4)))
)

In [0]:
ft_vendas_geral_df = (
    ft_pedidos_silver_df
        .join(ft_avaliacoes_pedidos_silver_df, "id_pedido", "left")
        .join(ft_itens_pedidos_silver_df, "id_pedido", "left")
        .join(ft_produtos_silver_df, "id_produto", "left")
        .join(
            dm_cotacao_dolar_silver_df,
            F.to_date("pedido_compra_timestamp") == F.col("data"),
            "left"
        )

        .select(
            # IDs e FKs
            F.col("id_pedido"),
            F.col("id_item"),
            F.col("id_consumidor").alias("fk_cliente"),
            F.col("id_produto").alias("fk_produto"),
            F.col("id_vendedor").alias("fk_vendedor"),
            F.to_date("pedido_compra_timestamp").alias("fk_tempo"),

            F.col("status").alias("status_pedido"),
            F.col("tempo_entrega_dias"),
            F.col("entrega_no_prazo"),

            # Valores BRL 
            F.col("preco_BRL").alias("valor_produto_brl").cast(DecimalType(12, 2)),
            F.col("preco_frete").alias("valor_frete_brl").cast(DecimalType(12, 2)),
            (
                (F.col("preco_BRL") + F.col("preco_frete"))
                .cast(DecimalType(12, 2))
            ).alias("valor_total_item_brl"),

            # Conversão para USD com proteção contra divisão por zero e nulo
            (
                F.when(F.col("cotacao_dolar").isNull(), None)
                 .when(F.col("cotacao_dolar") == 0, None)   # evita crash
                 .otherwise(F.col("preco_BRL") / F.col("cotacao_dolar"))
            ).cast(DecimalType(12,2)).alias("valor_produto_usd"),

            (
                F.when(F.col("cotacao_dolar").isNull(), None)
                 .when(F.col("cotacao_dolar") == 0, None)
                 .otherwise(F.col("preco_frete") / F.col("cotacao_dolar"))
            ).cast(DecimalType(12,2)).alias("valor_frete_usd"),

            (
                F.when(F.col("cotacao_dolar").isNull(), None)
                 .when(F.col("cotacao_dolar") == 0, None)
                 .otherwise(
                    (F.col("preco_BRL") + F.col("preco_frete")) /
                     F.col("cotacao_dolar")
                 )
            ).cast(DecimalType(12,2)).alias("valor_total_item_usd"),

            F.col("cotacao_dolar"),

            # Avaliação 
            F.col("avaliacao")
                .cast(DecimalType(3,2))
                .alias("avaliacao_pedido")
        )
)


display(ft_vendas_geral_df.limit(10))

ft_vendas_geral_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{gold_db_name}.ft_vendas_geral")


id_pedido,id_item,fk_cliente,fk_produto,fk_vendedor,fk_tempo,status_pedido,tempo_entrega_dias,entrega_no_prazo,valor_produto_brl,valor_frete_brl,valor_total_item_brl,valor_produto_usd,valor_frete_usd,valor_total_item_usd,cotacao_dolar,avaliacao_pedido
e481f51cbdc54678b7cc49136f2d6af7,1,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-02,entregue,8,Sim,29.99,8.72,38.71,9.48,2.76,12.24,3.1636,4.00
53cdb2fc8bc7dce0b6741e2150273451,1,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-24,entregue,14,Sim,118.70,22.76,141.46,31.69,6.08,37.77,3.7453,4.00
47770eb9100c2d0c44946d9cf07ec65d,1,41ce2a54c0b03bf3443c3d931a367089,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-08,entregue,9,Sim,159.90,19.22,179.12,42.63,5.12,47.75,3.7513,5.00
949d5b44dbf5de918fe9c16f97b45f8a,1,f88197465ea7920adcdbec7375364d82,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-18,entregue,14,Sim,45.00,27.20,72.20,13.73,8.30,22.02,3.2782,5.00
ad21c59c0840e6cb83a9ceb5573f8159,1,8ab97904e6daea8866dbdbc4fb7aad2c,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-13,entregue,3,Sim,19.90,8.72,28.62,null,null,null,null,5.00
a4591c265e18cb1dcee52889e2d8acc3,1,503740e9ca751ccdda7ba28e9ab8f608,060cb19345d90064d1015407193c233d,8581055ce74af1daba164fdbd55a40de,2017-07-09,entregue,17,Sim,147.90,27.36,175.26,44.97,8.32,53.29,3.2889,4.00
136cce7faa42fdb2cefd53fdc79a6098,1,ed0271e0b7da060a393796590e7b737a,a1804276d9941ac0733cfd409f5206eb,dc8798cbf453b7e0f98745e396cc5616,2017-04-11,faturado,null,Não Entregue,49.90,16.05,65.95,15.88,5.11,20.99,3.1418,2.00
6514b8ad8028c9f2cc2374ded245783f,1,9bdf08b4b3b52b5526ff42d37d47f222,4520766ec412348b8d4caa5e8a18c464,16090f2ca825584b5a147ab24aa30c86,2017-05-16,entregue,10,Sim,59.99,15.17,75.16,19.40,4.91,24.31,3.0918,5.00
76c6e866289321a7c93b82b54852dc33,1,f54a9f0e6b351c431402b8461ea51999,ac1789e492dcd698c5c10b97a671243a,63b9ae557efed31d1f7687917d248a8d,2017-01-23,entregue,10,Sim,19.90,16.05,35.95,6.30,5.08,11.38,3.1603,1.00
e69bfb5eb88e0ed6a785585b27e16dbf,1,31ad1d1b63eb9962463f764d4e6e0c9d,9a78fb9862b10749a117f7fc3c31f051,7c67e1448b00f6e969d365cea6b010ab,2017-07-29,entregue,18,Sim,149.99,19.77,169.76,47.69,6.29,53.98,3.1451,5.00


### 3.3 Criação da view gold.view_vendas_por_periodo

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_por_periodo AS
WITH juncao_vendas_tempo  AS (
    SELECT
        t.ano,
        t.trimestre,
        t.mes,
        t.dia,
        t.dia_da_semana_num,
        -- Conta o total de pedidos de diferentes ids
        CAST(COUNT(DISTINCT v.id_pedido) AS BIGINT) AS total_pedidos,
        CAST(COUNT(v.id_item) AS BIGINT) AS total_itens,

        CAST(SUM(v.valor_total_item_brl) AS DECIMAL(12,2)) AS receita_total_brl,
        CAST(SUM(v.valor_total_item_usd) AS DECIMAL(12,2)) AS receita_total_usd,
        CAST(AVG(v.valor_total_item_brl) AS DECIMAL(12,2)) AS ticket_medio_brl,
        CAST(AVG(v.avaliacao_pedido) AS DECIMAL(3,2)) AS avaliacao_media

    FROM medalhao.gold.ft_vendas_geral v
    INNER JOIN medalhao.gold.dm_tempo t
        ON v.fk_tempo = t.sk_tempo
    GROUP BY
        t.ano,
        t.trimestre,
        t.mes,
        t.dia,
        t.dia_da_semana_num
)
-- Para obter nome do mês
SELECT
    j.ano,
    j.trimestre,
    j.mes,
    d.mes_nome,
    j.dia,
    j.dia_da_semana_num,
    j.total_pedidos,
    j.total_itens,
    j.receita_total_brl,
    j.receita_total_usd,
    j.ticket_medio_brl,
    j.avaliacao_media
FROM juncao_vendas_tempo j
LEFT JOIN (
    SELECT DISTINCT ano, trimestre, mes, mes_nome
    FROM medalhao.gold.dm_tempo
) d
USING (ano, trimestre, mes)

ORDER BY ano, trimestre, mes, dia;

SELECT * 
FROM medalhao.gold.view_vendas_por_periodo
LIMIT 10;

ano,trimestre,mes,mes_nome,dia,dia_da_semana_num,total_pedidos,total_itens,receita_total_brl,receita_total_usd,ticket_medio_brl,avaliacao_media
2016,3,9,Setembro,4,1,1,2,136.23,null,68.12,1.00
2016,3,9,Setembro,5,2,1,1,75.06,22.94,75.06,1.00
2016,3,9,Setembro,13,3,1,0,null,null,null,1.00
2016,3,9,Setembro,15,5,1,3,143.46,43.05,47.82,1.00
2016,4,10,Outubro,2,1,1,1,109.34,33.69,109.34,1.00
2016,4,10,Outubro,3,2,8,8,595.14,184.08,74.39,3.67
2016,4,10,Outubro,4,3,63,69,11295.48,3508.21,163.70,3.69
2016,4,10,Outubro,5,4,47,56,9645.94,2981.49,172.25,3.21
2016,4,10,Outubro,6,5,51,58,9131.23,2826.65,157.44,3.37
2016,4,10,Outubro,7,6,46,52,8135.97,2532.31,156.46,3.33


#### 3.3.1 Consultas Analíticas

In [0]:
%sql
SELECT 
    case    
        when dia_da_semana_num = 1 then 'Domingo' 
        when dia_da_semana_num = 2 then 'Segunda-feira' 
        when dia_da_semana_num = 3 then 'Terça-feira' 
        when dia_da_semana_num = 4 then 'Quarta-feira' 
        when dia_da_semana_num = 5 then 'Quinta-feira' 
        when dia_da_semana_num = 6 then 'Sexta-feira' 
        when dia_da_semana_num = 7 then 'Sábado' 
    end AS dia_da_semana_nome,
    SUM(receita_total_brl) AS receita_total
FROM medalhao.gold.view_vendas_por_periodo
GROUP BY dia_da_semana_nome
ORDER BY receita_total DESC
LIMIT 1;


dia_da_semana_nome,receita_total
Segunda-feira,2609658.90


In [0]:
%sql
-- Retorna dados do último ano
WITH ultimo_ano AS (
    SELECT *
    FROM  medalhao.gold.view_vendas_por_periodo
    WHERE ano = (
        SELECT MAX(ano)
        FROM medalhao.gold.view_vendas_por_periodo
    )
)

SELECT 
    ano,
    mes,
    mes_nome,
    AVG(ticket_medio_brl) AS ticket_medio
FROM ultimo_ano
GROUP BY ano, mes, mes_nome
ORDER BY ticket_medio DESC
LIMIT 1;

ano,mes,mes_nome,ticket_medio
2018,9,Setembro,166.460000


### 3.4. Criação da gold.view_top_produto

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_top_produto AS
SELECT
    p.id_produto,
    p.categoria_produto,

    CAST(COUNT(v.id_item) AS BIGINT) AS quantidade_vendida,
    CAST(COUNT(DISTINCT v.id_pedido) AS BIGINT) AS total_pedidos,

    CAST(SUM(v.valor_produto_brl) AS DECIMAL(12,2)) AS receita_brl,
    CAST(SUM(v.valor_produto_usd) AS DECIMAL(12,2)) AS receita_usd,

    CAST(AVG(v.valor_produto_brl) AS DECIMAL(12,2)) AS preco_medio_brl,
    CAST(AVG(v.avaliacao_pedido) AS DECIMAL(3,2)) AS avaliacao_media,

    CAST(AVG(p.peso_produto_gramas) AS DECIMAL(8,2)) AS peso_medio_gramas

FROM medalhao.gold.ft_vendas_geral v
INNER JOIN medalhao.silver.ft_produtos p
    ON v.fk_produto = p.id_produto

GROUP BY
    p.id_produto,
    p.categoria_produto;

SELECT *
FROM medalhao.gold.view_top_produto
LIMIT 5;


id_produto,categoria_produto,quantidade_vendida,total_pedidos,receita_brl,receita_usd,preco_medio_brl,avaliacao_media,peso_medio_gramas
2b4609f8948be18874494203496bc318,beleza_saude,269,259,23527.13,7007.56,87.46,4.07,250.00
2c4930c4b284c7b99db2a4c52071a45e,moveis_decoracao,9,3,405.00,118.44,45.00,2.78,9050.00
c785fffd281ac9afffc38c8015455fd8,cama_mesa_banho,1,1,59.90,18.44,59.90,5.00,600.00
0ecade74f3c3fb56a9256d425e7d535c,informatica_acessorios,3,3,83.70,25.98,27.90,4.33,300.00
fbed7bca664c2d2d9a1412223f7c8aeb,automotivo,1,1,47.20,14.58,47.20,5.00,600.00


###3.5 Criação da view_vendas_produtos_esteticos


In [0]:
display(
    ft_produtos_silver_df.groupBy("categoria_produto").count().where(ft_produtos_silver_df.categoria_produto.like('fashion%'))
)

categoria_produto,count
fashion_calcados,173
fashion_bolsas_e_acessorios,849
fashion_underwear_e_moda_praia,53
fashion_roupa_masculina,95
fashion_roupa_feminina,27
fashion_esporte,19
fashion_roupa_infanto_juvenil,5


In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_fashion_performance AS
WITH vendas_fashion AS (
    SELECT
        t.ano,
        t.mes,
        p.categoria_produto,
        f.id_pedido,
        f.id_item,
        f.valor_produto_brl,
        f.valor_produto_usd,
        f.avaliacao_pedido
    FROM medalhao.gold.ft_vendas_geral f
    JOIN medalhao.gold.dm_tempo t ON f.fk_tempo = t.sk_tempo
    JOIN medalhao.silver.ft_produtos p ON f.fk_produto = p.id_produto
    WHERE p.categoria_produto LIKE 'fashion%'
)

SELECT
    ano,
    mes,
    categoria_produto,

    -- Contando total_pedidos e total_itens_vendidos
    CAST(COUNT(DISTINCT id_pedido) AS BIGINT) AS total_pedidos,
    CAST(COUNT(id_item) AS BIGINT) AS total_itens_vendidos,

    -- Calculando receitas
    CAST(SUM(valor_produto_brl) AS DECIMAL(12,2)) AS receita_total_brl,
    CAST(SUM(valor_produto_usd) AS DECIMAL(12,2)) AS receita_total_usd,

    CAST(AVG(valor_produto_brl) AS DECIMAL(12,2)) AS ticket_medio_brl,
    CAST(AVG(valor_produto_usd) AS DECIMAL(12,2)) AS ticket_medio_usd,
    CAST(AVG(avaliacao_pedido) AS DECIMAL(3,2)) AS avaliacao_media

FROM vendas_fashion
GROUP BY ano, mes, categoria_produto
-- Ordenando para facilitar a visualização
ORDER BY ano, mes, categoria_produto;

-- Visualizando a view criada
SELECT * FROM medalhao.gold.view_fashion_performance LIMIT 5;

ano,mes,categoria_produto,total_pedidos,total_itens_vendidos,receita_total_brl,receita_total_usd,ticket_medio_brl,ticket_medio_usd,avaliacao_media
2016,10,fashion_bolsas_e_acessorios,8,9,508.30,157.94,56.48,17.55,3.22
2016,10,fashion_calcados,1,1,29.99,9.28,29.99,9.28,5.00
2016,10,fashion_roupa_feminina,1,1,54.90,16.97,54.90,16.97,5.00
2016,10,fashion_roupa_masculina,1,1,24.90,7.70,24.90,7.70,1.00
2016,12,fashion_bolsas_e_acessorios,1,1,10.90,3.33,10.90,3.33,5.00
